In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

from data import read_history

In [ ]:
streaming_history = read_history()
print(len(streaming_history))
streaming_history.head()

In [ ]:
streaming_history["date"] = streaming_history["endTime"].dt.floor('d')
by_date = streaming_history.groupby("date").count()
by_date = by_date.sort_index()
by_date["weekday"] = by_date.index.weekday
by_date["week"] = by_date.index.week
by_date.head()

In [ ]:
week = 0
prev_week = by_date.iloc[0]["week"]
continuous_week = np.zeros(len(by_date)).astype(int)
sunday_dates = []
for i, (index, row) in enumerate(by_date.iterrows()):
    if row["week"] != prev_week:
        week += 1
        prev_week = row["week"]
    continuous_week[i] = week
by_date["continuous_week"] = continuous_week 
by_date.head()

## Transform into a heat-map like matrix

In [ ]:
songs = np.full((7, continuous_week.max()+1), np.nan)
for index, row in by_date.iterrows():
    songs[row["weekday"]][row["continuous_week"]] = row["trackName"]

In [ ]:
from datetime import timedelta

def get_monday(dt):
    prev_mon = dt - timedelta(dt.weekday())
    return prev_mon

previous_date = get_monday(streaming_history["endTime"].min())
previous_abbr = calendar.month_abbr[previous_date.month]
x_headers = [previous_abbr]
for i in range(continuous_week.max()):
    new_date = get_monday(streaming_history["endTime"].min() + timedelta(weeks=int(i)))
    new_abbr = calendar.month_abbr[new_date.month]
    if new_abbr != previous_abbr:
        previous_abbr = new_abbr
        x_headers.append(new_abbr)
    else:
        x_headers.append("")

In [ ]:
fig = plt.figure(figsize=(20,5))
ax = plt.subplot()
mask = np.isnan(songs)
sns.heatmap(songs, ax = ax, linewidths=2, 
            linecolor='white', mask=mask, cmap="Greens", 
            square=True, vmin=0, vmax=by_date["endTime"].max(), cbar=False)
y_desc = ["Mon", "", "Wed", "", "Fri", "", "Sun"]
ax.set_yticklabels(y_desc, rotation=0)
ax.set_xticklabels(x_headers, ha="left")
ax.set_facecolor("#ebedf0")
ax.xaxis.tick_top()
ax.set_title("My year on Spotify", fontsize=20,pad=40)
ax.tick_params(axis='both', which='both',length=0)
fig.patch.set_facecolor('white')